# Maji Ndogo Agriculture Analysis

**Integrated Project: Understanding Maji Ndogo's agriculture**

This notebook contains the implementation of all required functions for analyzing agricultural data in Maji Ndogo.

## Setup and Imports

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

## Data Loading and Cleaning

In [ ]:
# Create an engine for the database
engine = create_engine('sqlite:///Maji_Ndogo_farm_survey_small.db')

In [ ]:
# Test connection
with engine.connect() as connection:
    result = connection.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    for row in result:
        print(row)

In [ ]:
# SQL query to join all tables
sql_query = """
SELECT *
FROM geographic_features
LEFT JOIN weather_features USING (Field_ID)
LEFT JOIN soil_and_crop_features USING (Field_ID)
LEFT JOIN farm_management_features USING (Field_ID)
"""

In [ ]:
# Load data into DataFrame
with engine.connect() as connection:
    MD_agric_df = pd.read_sql_query(text(sql_query), connection)

In [ ]:
# Display the DataFrame
MD_agric_df

In [ ]:
# Drop Field_ID columns
MD_agric_df.drop(columns='Field_ID', inplace=True)

## Data Cleanup

In [ ]:
# Fix swapped column names
if 'Chosen_crop' in MD_agric_df.columns:
    MD_agric_df.rename(columns={'Chosen_crop': 'Crop_type'}, inplace=True)

if 'Soil_type' in MD_agric_df.columns:
    MD_agric_df.rename(columns={'Soil_type': 'Soil_Type'}, inplace=True)
    
if 'Soil_fertility' in MD_agric_df.columns:
    MD_agric_df.rename(columns={'Soil_fertility': 'Soil_Fertility'}, inplace=True)

# Fix spelling errors in crop types
if 'Crop_type' in MD_agric_df.columns:
    MD_agric_df['Crop_type'] = MD_agric_df['Crop_type'].str.lower().str.strip()
    crop_corrections = {
        'coffe': 'coffee',
        'cofee': 'coffee',
        'tee': 'tea',
        'te': 'tea',
        'maiz': 'maize',
        'mazie': 'maize',
        'weat': 'wheat',
        'whea': 'wheat',
        'rize': 'rice',
        'bananna': 'banana',
        'casava': 'cassava',
        'potatos': 'potato',
        'potatoes': 'potato'
    }
    MD_agric_df['Crop_type'] = MD_agric_df['Crop_type'].replace(crop_corrections)

# Fix negative elevation values
if 'Elevation' in MD_agric_df.columns:
    MD_agric_df['Elevation'] = MD_agric_df['Elevation'].abs()

## Final Data Checkup

In [ ]:
len(MD_agric_df['Crop_type'].unique())

In [ ]:
MD_agric_df['Elevation'].min()

In [ ]:
MD_agric_df['Annual_yield'].dtype

## Challenge 1: Uncovering Crop Preferences

In [ ]:
### START FUNCTION
def explore_crop_distribution(df, crop_filter):
    """
    Filter data by crop type and return mean Rainfall and Elevation.
    
    Args:
        df: DataFrame with agricultural data
        crop_filter: String value for the crop type to filter by
        
    Returns:
        tuple: (mean_rainfall, mean_elevation)
    """
    # Filter by crop type (case-insensitive)
    crop_filter_lower = crop_filter.lower()
    filtered_df = df[df['Crop_type'] == crop_filter_lower]
    
    # Calculate means
    mean_rainfall = filtered_df['Rainfall'].mean()
    mean_elevation = filtered_df['Elevation'].mean()
    
    return (mean_rainfall, mean_elevation)
### END FUNCTION

In [ ]:
# Test
explore_crop_distribution(MD_agric_df, "tea")

In [ ]:
explore_crop_distribution(MD_agric_df, "wheat")

## Challenge 2: Finding Fertile Grounds

In [ ]:
### START FUNCTION
def analyse_soil_fertility(df):
    """
    Group data by Soil_Type and return mean Soil_Fertility.
    
    Args:
        df: DataFrame with agricultural data
        
    Returns:
        pd.Series: Mean Soil_Fertility grouped by Soil_Type
    """
    # Group by Soil_Type and calculate mean Soil_Fertility
    result = df.groupby('Soil_Type')['Soil_Fertility'].mean()
    
    return result
### END FUNCTION

In [ ]:
# Test
analyse_soil_fertility(MD_agric_df)

## Challenge 3: Climate and Geography Analysis

In [ ]:
### START FUNCTION
def climate_geography_influence(df, column):
    """
    Group data by specified column and aggregate climate/geography metrics.
    
    Args:
        df: DataFrame with agricultural data
        column: Column name to group by
        
    Returns:
        pd.DataFrame: Aggregated data with Elevation, Min_temperature_C, 
                      Max_temperature_C, and Rainfall means
    """
    # Group by specified column and aggregate
    result = df.groupby(column)[['Elevation', 'Min_temperature_C', 'Max_temperature_C', 'Rainfall']].mean()
    
    return result
### END FUNCTION

In [ ]:
# Test
climate_geography_influence(MD_agric_df, 'Crop_type')

## Challenge 4: Advanced Sorting Techniques

In [ ]:
### START FUNCTION
def find_ideal_fields(df):
    """
    Find the top performing crop type based on above-average Standard_yield.
    
    Args:
        df: DataFrame with agricultural data
        
    Returns:
        str: Name of the top performing crop type
    """
    # Calculate average Standard_yield
    avg_yield = df['Standard_yield'].mean()
    
    # Filter fields with above-average Standard_yield
    above_avg_df = df[df['Standard_yield'] > avg_yield]
    
    # Group by Crop_type and count
    crop_counts = above_avg_df.groupby('Crop_type').size()
    
    # Sort in descending order
    crop_counts_sorted = crop_counts.sort_values(ascending=False)
    
    # Get the top crop type
    top_crop = crop_counts_sorted.index[0]
    
    return top_crop
### END FUNCTION

In [ ]:
# Test
type(find_ideal_fields(MD_agric_df))

In [ ]:
find_ideal_fields(MD_agric_df)

## Challenge 5: Advanced Filtering Techniques

In [ ]:
### START FUNCTION
def find_good_conditions(df, crop_type):
    """
    Filter DataFrame by crop type and specific conditions.
    
    Args:
        df: DataFrame with agricultural data
        crop_type: Type of crop to filter by
        
    Returns:
        pd.DataFrame: Filtered DataFrame
    """
    # Convert crop_type to lowercase for consistency
    crop_type_lower = crop_type.lower()
    
    # Calculate average Standard_yield
    avg_yield = df['Standard_yield'].mean()
    
    # Apply all filters
    filtered_df = df[
        (df['Crop_type'] == crop_type_lower) &
        (df['Standard_yield'] > avg_yield) &
        (df['Ave_temps'] >= 12) &
        (df['Ave_temps'] <= 15) &
        (df['Pollution_level'] < 0.0001)
    ]
    
    return filtered_df
### END FUNCTION

In [ ]:
# Test
find_good_conditions(MD_agric_df, "tea").shape

## Extra Pandas Nuggets

### Using df.query()

In [ ]:
# Example using query
MD_agric_df.query('Standard_yield > 0.5 and Soil_Type == "Loamy"')

In [ ]:
# Using variables with query
soil_types = ['Loamy', 'Sandy', 'Silt']
MD_agric_df.query('Soil_Type in @soil_types')

### Plotting Data with Pandas

In [ ]:
# Bar plot
MD_agric_df.groupby('Crop_type')['Standard_yield'].mean().plot(kind='bar')

In [ ]:
# Histogram
MD_agric_df['Standard_yield'].plot(kind='hist', bins=20)

In [ ]:
# Scatter plot
MD_agric_df.plot(kind='scatter', x='Pollution_level', y='Standard_yield')